
<img src="images/logo_UM.png" alt="logo Universidade do Minho" style="display: block; margin: auto; width: 25%" />
<p style="text-align: center; font-size: 12px">Licenciatura em Ciências da Computação</p>

<br/>

### <center>Unidade Curricular Projeto

<br/>

# <center>**Armazenamento de imagens e bases de dados vetoriais**
####   <center>Maio 2025</center>

<br/>

<br/>

**a53971** &ensp; Nuno Filipe Norberto Gonçalves de Oliveira

**a83783** &ensp; João Pinheiro Vitor

**a81820** &ensp; David João Oliveira Gonçalves

<br/>

---

<br/>
<br/>

## <center>Introdução

No âmbito da UC Projeto, foi-nos sugerida a criação de um sistema que recolha e armazene ficheiros de imagem num sistema de bases de dados vetorial, para posterior identificação de padrões de imagens.

No sentido de ir ao encontro do sugerido, decidimos criar um sistema de reconhecimento facial.

Este sistema permite ao utilizador fornecer uma imagem como *input*. O programa irá então identificar as faces presentes na imagem (caso existam) e como resultado apresenta as imagens, presentes na nossa base de dados, com as faces mais semelhantes ao *input*.

Ao longo deste documento iremos detalhar quais as tecnologias utilizadas, as etapas do desenvolvimento deste sistema e particularidades da sua implementação.

Por fim, faremos uma pequena demonstração do uso da aplicação.
<br><br>


### &ensp; Contextualização e motivação

O reconhecimento facial é um campo de investigação e desenvolvimento activo há várias décadas, com inúmeras aplicações já implementadas no nosso dia-a-dia. Desde o desbloqueio de smartphones até aos sistemas de vigilância, a tecnologia tem demonstrado o seu potencial e importância crescente.

No entanto, existem vários desafios, sejam eles na identificação de características das faces ou no armazenamento das imagens numa base de dados. Lidar com variações na aparência facial devido a diferentes condições de iluminação, ângulos de visão, expressões faciais e oclusões (como óculos ou máscaras) continua a ser um problema complexo. As dificuldades que as bases de dados tradicionais têm no armazenamento de imagens é também um entrave: as imagens podem ser ficheiros grandes, e armazenar um grande volume delas diretamente na base de dados pode levar a um crescimento exponencial do tamanho da mesma, o que pode levar a um baixo desempenho aplicacional.

É neste contexto que a utilização de bases de dados vetoriais surge como uma alternativa promissora. Ao representar imagens como vetores num espaço de alta dimensão, onde a distância entre vetores reflete a similaridade entre as imagens, torna-se possível realizar pesquisas mais eficientes e encontrar correspondências mesmo quando as imagens apresentam variações significativas.

Este projeto vai de encontro às tendências atuais na área de reconhecimento de padrões e análise de imagem, explorando a cooperação entre técnicas de visão computacional e sistemas de armazenamento. 

<br><br><br>

## <center>Tecnologias e bibiliotecas utilizadas

### &ensp; Base de dados

A seleção de uma base de dados vetorial que se adequasse às necessidades do nosso sistema foi uma etapa relevante no desenvolvimento, dada a sua importância para o armazenamento, indexação e pesquisa dos *embeddings* gerados.
<br>Após uma análise comparativa de três soluções – Pinecone, Qdrant e KDB.AI – a nossa escolha recaiu sobre [Qdrant](https://qdrant.tech/) como a ferramenta mais adequada para as necessidades específicas deste projeto.

Esta decisão foi fundamentada com base nos seguintes critérios:
- Qualidade da documentação

- Facilidade de configuração

- Possibilidade de uso localmente

- Persistência de dados (local)

Podemos ver na tabela seguinte os resultados da nossa avaliação:


| Critério | Pinecone | Qdrant | KBD.AI |
| --- | --- | ---| --- |
| **Qualidade da documentação** | Extensa, com exemplos, mas, por vezes, algo confusa | Extensa, com exemplos e bem estruturada | Algo confusa e com menos exemplos que Pinecone e Qdrant |
| **Facilidade de configuração** | Simples | Simples | Requer a instalação de mais bibliotecas |
| **Uso localmente** | Disponível mas limitado | Disponível | Disponível |
| **Persistência de dados (local)** | Não | Sim | Sim |

<br>

### &ensp; Linguagens e bibliotecas

Este projeto foi desenvolvido em Python (versão 3.11).

Entre as bibliotecas utilizadas, destacamos as seguintes:
- [Transformers](https://huggingface.co/docs/transformers/index): biblioteca de NPL's pré-treinadas, visão computacional, áudio e modelos multimodais para inferência e treino.

- [MTCNN (Multi-Task Cascaded Convolutional Neural Network)](https://mtcnn.readthedocs.io/en/latest/): biblioteca robusta para detecção facial, desenvolvida para detetar faces e os seus pontos de referência numa imagem, recorrendo a três CNNs. 

- [Pillow](https://pillow.readthedocs.io/en/stable/): biblioteca para processamento de imagens.

- [Qdrant-client](https://python-client.qdrant.tech/): biblioteca que permite utilizar a API de Qdrant.

- [Torch](https://pytorch.org/): biblioteca de *Machine Learning*



Foram também utilizadas outras bibliotecas para manipulação de dados numéricos e vetores.
<br><br>


### &ensp; Modelos *Vision Transformer*

A fase de transformação de imagens em *embeddings* é um passo fundamental para o bom desempenho do nosso sistema de reconhecimento facial, uma vez que a qualidade destas representações vetoriais impacta diretamente a capacidade do sistema em identificar padrões e realizar comparações eficazes. Dada a vasta gama de modelos de *Machine Learning* capazes de executar esta tarefa, a seleção de modelos adequados exigiu a definição de alguns critérios, que nos permitissem escolher aqueles que melhor se ajustassem aos objetivos do projeto.

Os critérios de selecção definidos foram os seguintes: 
- **tarefa para o qual o modelo foi treinado**: focamo-nos em modelos que foram especificamente treinados para a tarefa de *Image Feature Extraction*. Esta especialização garante que o modelo aprendeu a extrair características visuais relevantes das imagens, essenciais para a posterior comparação de similaridade facial.

- **tamanho e diversidade do dataset no qual foi treinado**: a capacidade de um modelo reconhecer e categorizar diferentes imagens depende da quantidade e diversidade dos dados nos quais foi treinado. [$^*$](https://developers.google.com/machine-learning/intro-to-ml/supervised)

- **resolução das imagens do dataset**: a resolução das imagens que compõem o nosso conjunto de dados situa-se predominantemente entre os 1024x768 e os 2048x1536 pixels. Tendo estes valores em consideração, a nossa abordagem consistiu em extrapolar um tamanho razoável para as faces presentes nas imagens, de maneira que houvesse informação suficiente para capturar os detalhes faciais de cada pessoa, e procurar modelos que tivessem sido treinados com resoluções de imagem que se aproximassem dessa escala facial estimada.

- **dimensão do *embedding* gerado**: a dimensão do vetor gerado deve ter em conta um equilíbrio entre a riqueza da informação codificada e a eficiência computacional. Um *embedding* de alta dimensão contém mais informação e detalhes acerca de uma imagem, mas também exige mais recursos de armazenamento e processamento.

Com base nestes critérios, realizamos uma pequena análise das diversas opções disponíveis na plataforma [HuggingFace](https://huggingface.co/models).

A nossa escolha recaiu sobre os modelos *google/vit-base-patch16-224-in21k* e *facebook/dino-vits16* pelos seguintes motivos:

- os dois modelos foram treinados para a tarefa pretendida

- ambos geram *embeddings* com a mesma dimensão - 768

- foram treinados com imagens de resolução 224x224 pixels, resolução que nos pareceu adequada para o sistema a implementar 

- o modelo da google foi treinado recorrendo a um dataset com 14 milhões de imagens e 21843 classes/categorias de objetos

- o modelo do facebook foi treinado recorrendo a um dataset com aproximadamente 1,3 milhões de imagens e 1000 classes/categorias de objetos

<br>


## <center>Implementação

### &ensp; Configuração Inicial


Para a execução bem-sucedida deste notebook, devemos certeficar-nos que os seguintes pré-requisitos são atendidos:

- **Disponibilidade das imagens**: as imagens a serem armazenadas na base de dados devem estar localizadas na diretoria **assets**.

- **Instalação de Docker**: a plataforma **[Docker](https://docs.docker.com/get-started/get-docker/)** deve estar devidamente instalada

- Todos os ***packages*** necessários para a execução do código devem ser previamente instalados


```bash
# Opcional
# Criar ambiente virtual recorrendo ao conda/miniconda
conda create -n my_env python=3.11 pip
conda activate my_env

# Ou recorrendo a virtualenv
python=3.11 -m venv venv
source venv/bin/activate

# Instalar packages
pip install -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cpu

# Transferência de outros módulos necessários
wget https://raw.githubusercontent.com/nfngo/LCC_Projecto_2024-2025/refs/heads/main/qdrant/utils.py
wget https://raw.githubusercontent.com/nfngo/LCC_Projecto_2024-2025/refs/heads/main/models/models.py

# Transferência da imagem docker de Qdrant
docker pull qdrant/qdrant

# Inicialização de container de Qdrant
!docker run -p 6333:6333 \
            -v $(pwd)/qdrant_storage:/qdrant/storage \
            qdrant/qdrant
```

<br>
Após garantirmos que temos todos os recursos instalados e a funcionar corretamente, importamos as bibliotecas necessárias e estabelecemos uma ligação com a base de dados.

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from PIL import Image
import os

import utils as qd
import models as mdl

In [ ]:
# Local dev
client = QdrantClient(host="localhost", port=6333)

De seguida, criamos uma **colecção**.

Uma colecção é um conjunto de pontos na qual podemos efetuar pesquisas. Estes pontos são constituídos por um vetor e um *payload*, que não é mais que uma *feature* de Qdrant que permite associar informação adicional aos vetores. Os vetores de cada ponto que pertença à mesma colecção devem ter a mesma dimensão e são comparados recorrendo a uma métrica de similaridade. 

Neste projeto, tendo em conta os modelos escolhidos para gerar *embeddings* (representações vetoriais das imagens), cada vetor será composto por 768 valores do tipo *float*.

Como métrica de similaridade, optamos pela similaridade por cosseno.

<br>

$$
similarity(A,B) = cos(\theta) = \frac{A \cdot B}{\|A\|\|B\|}
$$

<br>

Esta métrica permite calcular o cosseno do ângulo entre dois vetores, possibilitando saber de forma eficaz se os dois apontam na mesma direcção. Como resultado, obtém-se um valor no intervalo [-1, 1]. Valores próximos de 1 significam que os vetores têm direcções semelhantes e valores próximos de -1 indicam-nos que os vetores apontam em direcções opostas.

No contexto de *embeddings* de imagem que representam faces, a direcção representa as propriedades da face. Idealmente, duas imagens da face da mesma pessoa, se considerarmos diferentes condições de luz ou poses diferentes, devem apontar na mesma direcção. 

É importante, portanto, que seja possível "descartar" certas propriedades das imagens, tais como a luminosidade ou o contraste, que influenciam a magnitude dos vetores.
<br>Uma vez que no cáculo da similaridade por cosseno se faz uma normalização dos vetores pela sua magnitude, esta métrica torna-se ideal para o pretendido.

In [ ]:
collection = "image_collection"

# facebook/dino-vitb16 size: 768
# Vit Base Patch16 224 In21k size: 768
qd.create_collection(client, collection, 768, models.Distance.COSINE)

<br>

### &ensp; Processamento de Imagens

Ainda antes de passarmos à conversão das imagens para *embeddings*, é necessário armazenar informações de cada uma das imagens que pretendemos analisar, que devem estar guardadas na pasta "assets".

Como estrutura para armazenar esta informação, escolhemos uma lista de dicionários, que nos permite guardar, para cada imagem, os seguintes dados: nome da pessoa presente na imagem, nome do ficheiro e objeto de imagem.

De forma a agilizar o processo, o nome de cada ficheiro contém o nome da pessoa.

```python
data = [img_1, img_2, img_3...]

img_1 = {
        "name": ...,
        "file_name": ...,
        "img_obj": ...
    }
```

In [ ]:
current_directory = os.getcwd()
extensions = (".jpg", ".jpeg", ".webp")

def get_images_data(path):
    data = []
    for file in os.listdir(f"{path}"):
        if file.endswith(extensions):
            file_name = file.split(".")[0].lower().replace("_"," ")
            name = "".join(c for c in file_name if c.isalpha() or c == ' ')[:-1] 
            img_data = {
                "name": name,
                "file_name": file,
                "img_obj": Image.open(os.path.join(f"{path}",file))
            }
            data.append(img_data)
    
    return data

data = get_images_data(f"{current_directory}/assets")

In [ ]:
len(data), data

As etapas seguintes consistem em recortar a cara da pessoa presente em cada imagem e gerar o respetivo *embedding*.

Portanto, é necessário inicializar o modelo ViT que vamos utilizar e o modelo de detecção facial *MTCNN*.

In [ ]:
# Initialize models
# default model: vit-base-patch16-224-in21k
# dino = "dino-vitb16"
device, processor, model, detector = mdl.init_models()


Posteriormente, passamos à fase de recorte da face.

Nesta etapa, utilizamos o modelo *MTCNN* para obter a posição da face na imagem, através do método *detect_faces*. Este método devolve uma lista de dicionários, cada um deles contendo a posição da caixa delimitadora, pontos de referência faciais e pontuação de confiança de cada face detectada.

```python
# Exemplo
[
    {
        "box": [277, 90, 48, 63],
        "keypoints": {
            "nose": [303, 131],
            "mouth_right": [313, 141],
            "right_eye": [314, 114],
            "left_eye": [291, 117],
            "mouth_left": [296, 143]
        },
        "confidence": 0.9985
    }
]
```

Após a detecção facial, utilizamos a biblioteca Pillow para isolar e recortar as faces identificadas nas imagens.

Com o objetivo de otimizar a fase de *image embedding*, avaliamos as dimensões de cada recorte facial. Definimos uma dimensão alvo de 224x224 pixels. Caso a dimensão do recorte seja significativamente inferior ao pretendido (menos de um terço do tamanho alvo), descartamos o resultado por considerarmos que a informação pode ser insuficiente. Se a dimensão do recorte for superior ao tamanho alvo, realizamos um redimensionamento para dimensões próximas de 224x224 pixels, de forma a uniformizar os dados que vamos fornecer ao modelo ViT. 

Finalmente, cada recorte facial redimensionado (ou mantido, se as dimensões forem adequadas) é guardado como um ficheiro na pasta "faces", mantendo o mesmo nome do ficheiro original.

In [ ]:
# Create directory if needed
faces_path = "faces"
if not os.path.exists(faces_path):
    os.makedirs(faces_path)


destiny_path = f"{current_directory}/{faces_path}/"
for i, image_data in enumerate(data):
    mdl.process_image(image_data['img_obj'], 
                      detector,
                      destiny_path,
                      image_data['file_name'])

faces_data = get_images_data(f"{current_directory}/{faces_path}")

Em baixo, podemos ver os dados atualizados após o processamento das imagens.

In [ ]:
len(faces_data), faces_data

<br>

### &ensp; Processo de *Image Embedding*

Um dos nossos principais objetivos é extrair características das imagens (faces) e armazená-las na base de dados. Essas características não são mais que representações vetoriais das imagens (***embeddings***) que retêm o conteúdo visual das mesmas e são utilizadas para aumentar a performance de tarefas de computação visual, tais como detecção de objetos e classificação de imagens. 

<br/>
<img src="images/embedding.jpg" alt="image to vector embedding" style="display: block; margin: auto; width: 50%;" />
<br/>

Para extrair essa representação das características das imagens, iremos utilizar técnicas de ***Image Embedding***. Concretamente, faremos essa conversão recorrendo a modelos ***Vision Transformer (ViT)***.

De uma forma resumida, ViTs são modelos de *Machine Learning* avançados que permitem aos computadores "ver" e entender informação visual de uma maneira semelhante aos humanos. Baseiam-se numa arquitetura ***Transformer*** para processar imagens e extrair características significativas delas.

Para entender como funcionam os ViTs, vamos fazer uma analogia. Imaginemos que temos um puzzle com muitas peças diferentes. Para montar o puzzle, tipicamente olhamos para as peças individuais, para as suas formas e como se encaixam para formar uma imagem completa. 

ViTs funcionam da mesma maneira. Em vez de analizarem uma imagem completa, os *Vision Transformers* dividem a imagem em pedaços mais pequenos, chamados ***patches***. Cada um desses *patches* é como uma peça do puzzle. Após feita essa divisão, esses pedaços são analizados e processados pelos ViTs.

Ao ser feita essa análise, os ViTs identificam padrões importantes como, por exemplo, limites da imagem (*edges*), cores e texturas, e combina-os de maneira a entender a respetiva imagem de forma lógica, nítida e consistente.

Então, para cada imagem guardada na diretoria "faces", geramos o *embedding* correspondente, através da função *gen_embeddings*, e acrescentamos esta informação à nossa estrutura de dados. Para o efeito, acrescentamos o campo "img_embedding".

```python
{
    "name": ...,
    "file_name": ...,
    "img_obj": ...,
    "img_embedding": ...
}
```

In [ ]:
# Generate embeddings
embeddings = mdl.gen_embeddings(faces_data, processor, device, model)

In [ ]:
embeddings[0]

<br>

### &ensp; Armazenamento na base de dados

Esta etapa do nosso processo consiste no armazenamento estruturado da informação extraída de cada imagem na base de dados.

Conforme já tínhamos mencionado, em Qdrant guardamos informação sob a forma de "pontos".

```python
# Exemplo de um ponto
{
    "id": 129,
    "vector": [0.1, 0.2, 0.3, 0.4],
    "payload": {"color": "red"},
}
```

No contexto do nosso sistema de reconhecimento facial, cada imagem processada será representada como um ponto. O *embedding* gerado para a face detectada na imagem (presente no campo 'img_embedding' da nossa estrutura de dados) será armazenado no campo 'vector' do ponto.

Adicionalmente, para preservar informações relevantes sobre cada imagem, utilizaremos o campo 'payload' do ponto. Neste campo, armazenaremos um dicionário contendo as restantes informações associadas à imagem, nomeadamente o nome da pessoa identificada ('name') e o nome do ficheiro da imagem original ('file_name').

In [ ]:
# Save embeddings to DB
for img_data in embeddings:
    qd.insert_image_embedding(client, collection, img_data)

Uma possível representação 2D da nossa base de dados é a seguinte:

<br/>
<img src="images/print_BD.jpg" alt="DB 2D representation" style="display: block; margin: auto; width: 90%;" />
<br/>


<br>

### &ensp; Procura por similaridade

Finalmente, chegamos ao ponto central da nossa aplicação: a busca por similaridade facial.

Dado uma imagem como *input*, o objetivo é identificar as faces presentes, gerar os seus respetivos *embeddings* e, subsequentemente, procurar da nossa base de dados as imagens de faces mais semelhantes às encontradas na imagem original.

O processo inicia-se, portanto, com a detecção de faces na imagem fornecida e a geração dos *embeddings* correspondentes.

Esta etapa crucial é implementada na função *gen_embedding_img_to_search*, que tem como resultado duas listas: uma contendo os *embeddings* das faces detectadas e outra com os objetos de imagem dessas mesmas faces.

In [ ]:
img_to_search, searched_faces = mdl.gen_embedding_img_to_search(f"{current_directory}/img_to_search/taskmaster.jpg", processor, device, model, detector)
img_to_search, searched_faces

In [ ]:
len(img_to_search), len(img_to_search[0])

Em seguida, para cada *embedding* das faces detectadas na imagem fornecida, realizamos uma busca pelas X faces mais semelhantes na nossa base de dados. Os resultados dessas buscas são então armazenados na variável *nearest_results*.

Como veremos, esta variável contém a resposta a cada consulta efetuada à base de dados, apresentando para cada *embedding*, uma lista de pontos correspondentes aos resultados mais próximos.

São de particular interesse para nós os campos 'score' e 'payload', que nos permitem saber a pontuação de similaridade, o nome da pessoa a quem a face pertence e o nome do ficheiro de imagem que lhe é associado.

In [ ]:
# Search top X similar results
top = 3
nearest_results = []
for img in img_to_search:
    nearest = qd.get_top_x_similar_images(client, collection, top, img)
    nearest_results.append(nearest)

nearest_results

Por fim, implementamos a função *print_results* que nos permite consultar os resultados obtidos de uma forma mais legível e gráfica.

In [ ]:
def print_results(results):
    msg = f"{len(results)} faces were found in the provided image" if len(results) > 1 else f"{len(results)} face was found in the provided image"
    print(msg)
    for j, result in enumerate(results):
        # print(f"{j}: {result.points}")
        points = result.points
        print(f"Face {j+1}:")
        searched_faces[j].show()
        for i in range(len(points)):
            name = points[i].payload['name']
            score = points[i].score
            print(f"Result #{i+1}: {name} was diagnosed with {score * 100} confidence")
            print(f"This image score was {score}")
            Image.open(f"faces/{points[i].payload['file_name']}").show()
            print("-" * 50)
            print()

print_results(nearest_results)

Com o objetivo de, no futuro, aumentar as funcionalidades da nossa aplicação, deixamos também disponíveis funções que nos permitem eliminar e consultar pontos da base de dados. 

In [ ]:
res = qd.get_points_from_collection(client, collection, 5)
res

In [ ]:
id = res[0][0].id
qd.delete_points_from_collection(client, collection, [id])

<br>

### &ensp; Aplicação *Facial Recognition System*

A última etapa do nosso projeto consistiu em implementar uma aplicação que permitisse, de uma forma simples e intuitiva, fazer uma procura por similaridade entre uma imagem de uma pessoa fornecida pelo utilizador e os dados armazenados na nossa base de dados.

Para o efeito, utilizamos a framework de desenvolvimento aplicacional [Streamlit](https://streamlit.io/).

In [ ]:
%%writefile search_image.py

from qdrant_client import QdrantClient
import os
import streamlit as st
import torch

import utils as qd
import models as mdl

torch.classes.__path__ = []  # Neutralizes the path inspection

# Local dev
client = QdrantClient(host="localhost", port=6333)

collection = "image_collection"

current_directory = os.getcwd()

# Initialize models
device, processor, model, detector = mdl.init_models()

st.title("Facial Recognition System")
st.markdown("Upload images with different faces and you'll get the most similar ones from our database.")

uploaded_file = st.file_uploader(label = "Upload some image",
                                 type=["jpg", "jpeg", "webp"])

# Search top X similar results
top = st.radio(
    "How many search results do you want?",
    [1, 3, 5],
)

with st.spinner("Waiting for results..."):
    if uploaded_file:
        img_to_search, searched_faces = mdl.gen_embedding_img_to_search(uploaded_file, processor, device, model, detector)


        nearest_results = []
        for img in img_to_search:
            nearest = qd.get_top_x_similar_images(client, collection, top, img)
            nearest_results.append(nearest)

        st.markdown("## Results:")

        def print_results(results):
            msg = f"{len(results)} faces were found in the provided image" if len(results) > 1 else f"{len(results)} face was found in the provided image"
            st.markdown(msg)

            for j, result in enumerate(results):
                points = result.points
                st.markdown(f"Face {j+1}:")
                st.image(searched_faces[j], width = 200)
                for i in range(len(points)):
                    name = points[i].payload['name']
                    score = points[i].score
                    st.markdown(f"Result #{i+1}: {name} was diagnosed with {score * 100} confidence")
                    st.markdown(f"This image score was {score}")
                    st.image(f"faces/{points[i].payload['file_name']}")
                    st.markdown("-" * 50)


        print_results(nearest_results)

In [ ]:
!streamlit run search_image.py

<br>

---

<br>

## <center>Conclusão

Este projeto constituiu uma experiência de aprendizagem valiosa, proporcionando-nos a oportunidade de aprofundar conhecimentos em diversas áreas tecnológicas. Tivemos a possibilidade de trabalhar com bases de dados vetoriais, explorando a sua estrutura e potencial para armazenamento e pesquisa de dados complexos. A utilização de modelos de *Machine Learning* para a transformação de imagens em *embeddings* permitiu-nos compreender o poder da representação vetorial na análise visual. Adicionalmente, aplicamos técnicas de processamento de imagem para preparar os dados e desenvolvemos uma aplicação web, recorrendo a bibliotecas e frameworks especializados, consolidando assim as nossas competências de desenvolvimento.

Do nosso ponto de vista, o objetivo inicial de conceber um sistema de recolha e armazenamento de imagens num sistema de bases de dados vetorial, com a finalidade de identificar padrões visuais, foi efetivamente alcançado. O sistema implementado demonstra a capacidade de processar imagens, extrair características relevantes e armazená-las de forma que seja possível fazer pesquisas por imagens similares.

Contudo, reconhecemos que há muito a explorar no sentido de refinar e expandir este projeto. Para evoluir para um sistema mais robusto, abrangente e fiável, poderíamos seguir por vários caminhos de desenvolvimento. Uma direção promissora seria treinar os modelos de extração de características. Ajustar os modelos aos nossos dados e à tarefa particular de reconhecimento facial poderia otimizar significativamente a qualidade dos *embeddings* gerados e, consequentemente, a precisão do sistema. Outra melhoria crucial seria tornar a aplicação mais interativa e flexível, permitindo ao utilizador fornecer uma imagem representativa de um padrão específico a ser procurado na base de dados. Esta funcionalidade expandiria significativamente a usabilidade do sistema para cenários de pesquisa mais direcionados.